In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import json
import pandas as pd
import numpy as np
# import spacy
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_colwidth', None)

In [3]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict

# Read Data

In [4]:
rev_kp_dataset_df = pd.read_pickle("../data/training/train_data.pkl")

In [5]:
rev_kp_dataset_df

,index,level_1,domain,categories_list,categories,name,business_id,sentences,opinions_x,aspects_x,...,opinion_phrases_y,key_point,predicted_WA_y,label,aspects_similarity,aspects_x_len,aspects_y_len,id_pair,sentiments_x,sentiments_y
1,487383,2,Hotels,"['Event Planning & Services', 'Venues & Event Spaces', 'Hotels', 'Hotels & Travel']","Event Planning & Services, Venues & Event Spaces, Hotels, Hotels & Travel",Sheraton Centre Toronto Hotel,M7FOXsuEuRbwEfxUrNKv5w,Free WiFi in lobby,[free],[wifi],...,[very fast wifi],Wifi was very fast ~ 20MBps .,0.5944222211837769,1.0,NaN,NaN,NaN,487383_2##702558_5,[positive],[positive]
3,487385,0,Hotels,"['Event Planning & Services', 'Venues & Event Spaces', 'Hotels', 'Hotels & Travel']","Event Planning & Services, Venues & Event Spaces, Hotels, Hotels & Travel",Sheraton Centre Toronto Hotel,M7FOXsuEuRbwEfxUrNKv5w,Been staying at this hotel for years now and still enjoy the place .,[enjoy],[place],...,[family friendly place],"Relaxed , quiet , family friendly place .",0.5604457855224609,1.0,NaN,NaN,NaN,487385_0##777430_1,[positive],[positive]
4,487385,0,Hotels,"['Event Planning & Services', 'Venues & Event Spaces', 'Hotels', 'Hotels & Travel']","Event Planning & Services, Venues & Event Spaces, Hotels, Hotels & Travel",Sheraton Centre Toronto Hotel,M7FOXsuEuRbwEfxUrNKv5w,Been staying at this hotel for years now and still enjoy the place .,[enjoy],[place],...,[family oriented place],Definitely a family oriented place .,0.43907630443573,1.0,NaN,NaN,NaN,487385_0##511294_4,[positive],[positive]
27,487386,3,Hotels,"['Event Planning & Services', 'Venues & Event Spaces', 'Hotels', 'Hotels & Travel']","Event Planning & Services, Venues & Event Spaces, Hotels, Hotels & Travel",Sheraton Centre Toronto Hotel,M7FOXsuEuRbwEfxUrNKv5w,Staff was friendly .,[friendly],[staff],...,[above and beyond staff],Staff goes above and beyond .,0.5060063004493713,1.0,NaN,NaN,NaN,487386_3##922097_1,[positive],[positive]
28,487386,3,Hotels,"['Event Planning & Services', 'Venues & Event Spaces', 'Hotels', 'Hotels & Travel']","Event Planning & Services, Venues & Event Spaces, Hotels, Hotels & Travel",Sheraton Centre Toronto Hotel,M7FOXsuEuRbwEfxUrNKv5w,Staff was friendly .,[friendly],[staff],...,[amazing staff],Amazing staff & customer service .,0.5119611024856567,1.0,NaN,NaN,NaN,487386_3##884704_1,[positive],[positive]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50051,880945,1,Arts & Entertainment,"['Performing Arts', 'Arts & Entertainment']","Performing Arts, Arts & Entertainment",Rock of Ages,_F3AMoo_zdl-he384ISQbw,Ushers are all standby to guide you to the seats .,[standby],[ushers],...,[friendly and attentive concessions],Concessions are friendly and attentive .,0.48531198501586914,0.0,NaN,NaN,NaN,880945_1##999990_2,[negative],[positive]
50052,929246,2,Arts & Entertainment,"['Cinema', 'Arts & Entertainment']","Cinema, Arts & Entertainment",Cinemark Mesa 16,lqCJiHlxzRpuYt-u_XQUvg,Management was horrible and I 'm beyond disappointed with how rude and unprofessional they were !,[horrible],[management],...,[friendly and attentive concessions],Concessions are friendly and attentive .,0.48531198501586914,0.0,NaN,NaN,NaN,929246_2##999990_2,[negative],[positive]
50053,901588,0,Arts & Entertainment,"['Casinos', 'Nightlife', 'Arts & Entertainment', 'Bars', 'Adult Entertainment']","Casinos, Nightlife, Arts & Entertainment, Bars, Adult Entertainment",Playboy Club,3g8zzg_5__kXMUM-8F6V1w,Sexy dealers .,[sexy],[dealers],...,[friendly and attentive concessions],Concessions are friendly and attentive .,0.48531198501586914,0.0,NaN,NaN,NaN,901588_0##999990_2,[positive],[positive]
50054,842226,3,Arts & Entertainment,"['Arts & Entertainment', 'Arcades', 'Active Life', 'Venues & Event Spaces', 'Party & Event Planning', 'Amusement Parks', 'Event Planning & Services', 'Mini Golf', 'Laser Tag']","Arts & Entertainment, Arcades, Active Life, Venues & Event Spaces, Party & Event Planning, Amusement Parks, E

In [6]:
rev_kp_dataset_df = rev_kp_dataset_df.rename(columns={'attributes': 'topic', 'sentences': 'text'})
rev_kp_dataset_df['topic'] = rev_kp_dataset_df['topic'].apply(lambda x: ','.join(x))

# Prepare Contrastive Input

In [7]:
rev_kp_dataset_df = rev_kp_dataset_df.rename(columns={'text': 'full_comment', 'key_point': 'full_keypoint'})

In [8]:
def prepare_contrastive_input(row):
    return row.iloc[0][0] + ' <SEP> ' + row.iloc[1][0]

In [9]:
rev_kp_dataset_df['comment'] = rev_kp_dataset_df[['opinion_phrases_x', 'sentiments_x']].apply(prepare_contrastive_input, axis=1)
rev_kp_dataset_df['keypoint'] = rev_kp_dataset_df[['opinion_phrases_y', 'sentiments_y']].apply(prepare_contrastive_input, axis=1)
rev_kp_dataset_df[['comment', 'keypoint']]

,comment,keypoint
1,free wifi <SEP> positive,very fast wifi <SEP> positive
3,enjoy place <SEP> positive,family friendly place <SEP> positive
4,enjoy place <SEP> positive,family oriented place <SEP> positive
27,friendly staff <SEP> positive,above and beyond staff <SEP> positive
28,friendly staff <SEP> positive,amazing staff <SEP> positive
...,...,...
50051,standby ushers <SEP> negative,friendly and attentive concessions <SEP> positive
50052,horrible management <SEP> negative,friendly and attentive concessions <SEP> positive
50053,sexy dealers <SEP> positive,friendly and attentive concessions <SEP> positive
50054,freaking awesome staff <SEP> positive,friendly and attentive concessions <SEP> positive


# Business Category Split

Create train/val split set on each category. Train on full, val on part of train

In [10]:
rev_kp_dataset_df.columns

Index(['index', 'level_1', 'domain', 'categories_list', 'categories', 'name',
       'business_id', 'full_comment', 'opinions_x', 'aspects_x',
       'opinion_phrases_x', 'sid', 'topic', 'opinion_aspect_pos', 'aspect_pos',
       'num_of_token', 'predicted_WA_x', 'id_x', 'id_y', 'aspects_y',
       'opinions_y', 'opinion_phrases_y', 'full_keypoint', 'predicted_WA_y',
       'label', 'aspects_similarity', 'aspects_x_len', 'aspects_y_len',
       'id_pair', 'sentiments_x', 'sentiments_y', 'comment', 'keypoint'],
      dtype='object')

In [11]:
dataset = Dataset.from_pandas(rev_kp_dataset_df[['topic', 'full_comment', 'full_keypoint', 'comment', 'keypoint', 'domain', 'predicted_WA_x', 'predicted_WA_y',
                                                 'label']]).shuffle(seed=42)

In [12]:
from sklearn.model_selection import KFold, GroupKFold
from sklearn.model_selection import GroupShuffleSplit, ShuffleSplit

k = 6
groups = dataset['topic']
gss = GroupShuffleSplit(n_splits=1, train_size=0.85, random_state=32)

for train_idx, val_idx in gss.split(dataset, groups=groups):
    dataset.set_format("pandas")
    d = dataset[val_idx][:]
    print("VAL CLASSES: ", len(d['topic'].unique()))
    dataset.set_format(None)
    
    # Define data loaders for training and testing data in this fold
    train_dataset = dataset
    val_dataset = Dataset.from_dict(dataset[val_idx])
    
    train_dataset.set_format("pandas")
    d = train_dataset[:]
    print(len(d['topic'].unique()))
    train_dataset.set_format(None)

    val_dataset.set_format("pandas")
    d = val_dataset[:]
    print(len(d['topic'].unique()))
    val_dataset.set_format(None)

/home/an/anaconda3/envs/deep_learning_env_2/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


VAL CLASSES:  2
13
2


In [13]:
full_datasets = DatasetDict({
        "train": train_dataset,
        "valid": val_dataset})

Create datasets on each business category

## In-Catgegory Split

In [14]:
domain_datasets = DatasetDict()
for domain in rev_kp_dataset_df['domain'].unique():
    domain_datasets[domain] = full_datasets.filter(lambda examples: examples['domain'] == domain)

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [15]:
from sklearn.model_selection import KFold, GroupKFold
from sklearn.model_selection import GroupShuffleSplit, ShuffleSplit

for domain in domain_datasets.keys():
    domain_dataset = domain_datasets[domain]
    if len(domain_dataset['valid']) == 0:
        groups = domain_dataset['train']['topic']
        gss = GroupShuffleSplit(n_splits=1, train_size=0.85, random_state=32)
        for train_idx, val_idx in gss.split(domain_dataset['train'], groups=groups):
            domain_dataset['train'].set_format(None)
            new_val_dataset = Dataset.from_dict(domain_dataset['train'][val_idx])
            domain_dataset['valid'] = new_val_dataset

In [16]:
domain_datasets

DatasetDict({
    Hotels: DatasetDict({
        train: Dataset({
            features: ['topic', 'full_comment', 'full_keypoint', 'comment', 'keypoint', 'domain', 'predicted_WA_x', 'predicted_WA_y', 'label', '__index_level_0__'],
            num_rows: 14562
        })
        valid: Dataset({
            features: ['topic', 'full_comment', 'full_keypoint', 'comment', 'keypoint', 'domain', 'predicted_WA_x', 'predicted_WA_y', 'label', '__index_level_0__'],
            num_rows: 519
        })
    })
    Arts & Entertainment: DatasetDict({
        train: Dataset({
            features: ['topic', 'full_comment', 'full_keypoint', 'comment', 'keypoint', 'domain', 'predicted_WA_x', 'predicted_WA_y', 'label', '__index_level_0__'],
            num_rows: 8539
        })
        valid: Dataset({
            features: ['topic', 'full_comment', 'full_keypoint', 'comment', 'keypoint', 'domain', 'predicted_WA_x', 'predicted_WA_y', 'label', '__index_level_0__'],
            num_rows: 550
        })
  

In [17]:
for domain in rev_kp_dataset_df.domain.unique():
    
    datasets = domain_datasets[domain]
    contrastive_datasets = datasets
    
    contrastive_datasets.set_format("pandas")
    training_df = contrastive_datasets['train'][:]
    valid_df = contrastive_datasets['valid'][:]
    
    outdir = f'./training_material/siamese-data/in-category/{domain}'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    df = training_df[training_df['domain'] == domain].copy()
    # df['keypoint'] = df.apply(lambda x: x['topic'] + ' <SEP> ' + x['key_point'], axis=1)
    df['label'] = df.label.apply(lambda x: int(x))
    df[['comment', 'keypoint', 'domain', 'label']].to_csv(f'{outdir}/training_df_contrastive.csv')

    df = valid_df[valid_df['domain'] == domain].copy()
    # df['keypoint'] = df.apply(lambda x: x['topic'] + ' <SEP> ' + x['key_point'], axis=1)
    df['label'] = df.label.apply(lambda x: int(x))
    df[['comment', 'keypoint', 'domain', 'label']].to_csv(f'{outdir}/valid_df_contrastive.csv')

### Create separate sets of comments/KPs

In [18]:
from comment_kp_split import *

In [19]:
for domain in rev_kp_dataset_df['domain'].unique():
    datasets = domain_datasets[domain]
    contrastive_datasets = datasets
    contrastive_datasets.set_format("pandas")
    contrastive_datasets = contrastive_datasets.rename_columns({'full_keypoint': 'full_key_point', 'keypoint': 'key_point'})
    
    i = 0
    curr_topic = ""
    sub_datasets = contrastive_datasets.filter(lambda examples: examples['domain'] == domain)
    outdir = f'./training_material/siamese-data/in-category/{domain}'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    for subset, subset_save in zip(['train', 'valid'], ['train', 'dev']):
        comments_df, keypoints_df, labels_df = create_arg_kp_label_df(sub_datasets, subset)
        comments_df.to_csv(f"{outdir}/comments_{subset_save}.csv", index=False)
        keypoints_df.to_csv(f"{outdir}/key_points_{subset_save}.csv", index=False)
        labels_df.to_csv(f"{outdir}/labels_{subset_save}.csv", index=False)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Out-of-Category Split

In [20]:
domain_datasets = DatasetDict()
for domain in rev_kp_dataset_df['domain'].unique():
    domain_datasets[domain] = full_datasets.filter(lambda examples: examples['domain'] != domain)

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [21]:
for domain in rev_kp_dataset_df.domain.unique():
    
    datasets = domain_datasets[domain]
    contrastive_datasets = datasets
    
    contrastive_datasets.set_format("pandas")
    training_df = contrastive_datasets['train'][:]
    valid_df = contrastive_datasets['valid'][:]
    
    outdir = f'./training_material/siamese-data/out-of-category/{domain}'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    df = training_df[training_df['domain'] != domain].copy()
    df['label'] = df.label.apply(lambda x: int(x))
    df[['comment', 'keypoint', 'domain', 'label']].to_csv(f'{outdir}/training_df_contrastive.csv')

    df = valid_df[valid_df['domain'] != domain].copy()
    df['label'] = df.label.apply(lambda x: int(x))
    df[['comment', 'keypoint', 'domain', 'label']].to_csv(f'{outdir}/valid_df_contrastive.csv')

### Create separate sets of comments/KPs

In [22]:
from comment_kp_split import *

In [23]:
for domain in rev_kp_dataset_df['domain'].unique():
    datasets = domain_datasets[domain]
    contrastive_datasets = datasets
    contrastive_datasets.set_format("pandas")
    contrastive_datasets = contrastive_datasets.rename_columns({'full_keypoint': 'full_key_point', 'keypoint': 'key_point'})
    
    i = 0
    curr_topic = ""
    sub_datasets = contrastive_datasets.filter(lambda examples: examples['domain'] != domain)
    outdir = f'./training_material/siamese-data/out-of-category/{domain}'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    for subset, subset_save in zip(['train', 'valid'], ['train', 'dev']):
        comments_df, keypoints_df, labels_df = create_arg_kp_label_df(sub_datasets, subset)
        comments_df.to_csv(f"{outdir}/comments_{subset_save}.csv", index=False)
        keypoints_df.to_csv(f"{outdir}/key_points_{subset_save}.csv", index=False)    
        labels_df.to_csv(f"{outdir}/labels_{subset_save}.csv", index=False)    

  0%|          | 0/44 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/51 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/52 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]